In [73]:
import pandas as pd
import rich
import numpy as np
import subprocess
import shlex
import zipfile

from datasets import Dataset
from databench_eval import Runner, Evaluator
import openai_test

In [74]:
def load_dataset(type = 'sample'):
    ibm_hr = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/066_IBM_HR/{type}.parquet")
    tripadvisor = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/067_TripAdvisor/{type}.parquet")
    worldBank = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/068_WorldBank_Awards/{type}.parquet")
    taxonomy = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/069_Taxonomy/{type}.parquet")
    openfoodfacts = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/070_OpenFoodFacts/{type}.parquet")
    col = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/071_COL/{type}.parquet")
    admissions = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/072_Admissions/{type}.parquet")
    med_cost = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/073_Med_Cost/{type}.parquet")
    lift = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/074_Lift/{type}.parquet")
    mortality = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/075_Mortality/{type}.parquet")
    nba = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/076_NBA/{type}.parquet")
    gestational = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/077_Gestational/{type}.parquet")
    fires = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/078_Fires/{type}.parquet")
    coffee = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/079_Coffee/{type}.parquet")
    books = pd.read_parquet(f"/Users/tonypiacentini/csc-306-Project-4/src/data/080_Books/{type}.parquet")
    data = pd.DataFrame(columns = ['DataSet','DataRaw'])
    data.loc[len(data)] = ['066_IBM_HR', ibm_hr.to_csv(index=False)]
    data.loc[len(data)] = ['067_TripAdvisor', tripadvisor.to_csv(index=False)]
    data.loc[len(data)] = ['068_WorldBank_Awards', worldBank.to_csv(index=False)]
    data.loc[len(data)] = ['069_Taxonomy', taxonomy.to_csv(index=False)]
    data.loc[len(data)] = ['070_OpenFoodFacts', openfoodfacts.to_csv(index=False)]
    data.loc[len(data)] = ['071_COL', col.to_csv(index=False)]
    data.loc[len(data)] = ['072_Admissions', admissions.to_csv(index=False)]
    data.loc[len(data)] = ['073_Med_Cost', med_cost.to_csv(index=False)]
    data.loc[len(data)] = ['074_Lift', lift.to_csv(index=False)]
    data.loc[len(data)] = ['075_Mortality', mortality.to_csv(index=False)]
    data.loc[len(data)] = ['076_NBA', nba.to_csv(index=False)]
    data.loc[len(data)] = ['077_Gestational', gestational.to_csv(index=False)]
    data.loc[len(data)] = ['078_Fires', fires.to_csv(index=False)]
    data.loc[len(data)] = ['079_Coffee', coffee.to_csv(index=False)]
    data.loc[len(data)] = ['080_Books', books.to_csv(index=False)]
    data = data.set_index('DataSet')
    return data

In [75]:
#d = load_dataset()
#d.head()

In [76]:
#print(d['DataRaw'])

In [77]:
#for name, raw in d.itertuples():
#    print(name)


In [78]:
#query = "Does the dataset contain any review that more than forty users have labeled as helpful?"
#dataframe = d.loc['067_TripAdvisor']
#result_df1 = openai_test.response_test(query, dataframe['DataRaw'])

In [79]:
#print(result_df1)

In [80]:
#test_qa = pd.read_csv("/Users/tonypiacentini/csc-306-Project-4/src/data/test_qa.csv")  
#test_qa.tail()

In [92]:
def get_answers(test_qa, data):
    runner = 0
    last_name = ""
    final = []
    for name, query in test_qa.itertuples():
        if(runner % 10 == 0):
            print(runner)
        dataframe = data.loc[name]
        result = openai_test.response_test(query, dataframe['DataRaw'])
        result = result.replace("{", "").replace("}", "")
        final.append(result)
        runner += 1
    return final

In [82]:
#answers = get_answers(test_qa, d)

In [83]:
def clean(final: list):
    ans_list = []
    for i in final:
        ans = i.split(":")[1]
        ans_list.append(ans.replace("\n", "").replace("'", "").replace("```","").replace('"',""))#.replace("",""))
    return ans_list


In [84]:
import ast

def convert_string(s):
    """Convert a string to a list, boolean, or float if possible."""
    s = s.strip()
    # Try converting to a boolean first
    if s.lower() == "true" or s.lower() == "yes" or s.lower() == "y":
        return True
    elif s.lower() == "false" or s.lower() == "no" or s.lower() == "n":
        return False
    try:
        int_val = int(s)
        if str(int_val) == s:  # Ensure it's a true integer (not "3.14")
            return int_val
    except ValueError:
        pass
    # Try converting to a float
    try:
        return float(s)
    except ValueError:
        pass

    
    if(len(s) != 0):
        list_val = []
        if s[0] == "[" and s[-1] == "]":
            for i in s.replace("[","").replace("]","").split(","):
                list_val.append(convert_string(i))
            return list_val

    # If no conversion is possible, return the original string
    return s


In [102]:
def reformat_answers(answers):
    valid = []
    clean_answers = clean(answers)
    for i in clean_answers:
        valid.append(convert_string(i))
    return valid


In [118]:
test_qa = pd.read_csv("/Users/tonypiacentini/csc-306-Project-4/src/data/test_qa.csv")  
test_qa = test_qa.set_index('dataset')

In [87]:
data_lite = load_dataset()
data_all = load_dataset(type = "all")

In [119]:
answers_lite = get_answers(test_qa, data_lite)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520


In [ ]:
#answers_all = get_answers(test_qa, data_all)

In [120]:
reformatted_lite = reformat_answers(answers_lite)
#reformatted_all = reformat_answers(answers_all)

code to set up the evaluator with the correct answers to the test_qa.csv. This code, the answers, as well as the answer types comes from the databench_eval github under their competition_baseline.py file. Using this we are able to correctly evaluate using the test_qa questions.

In [115]:
qa_df = pd.DataFrame()
qaa = test_qa
# Read the answers from the .txt files into separate lists
with open("/Users/tonypiacentini/csc-306-Project-4/src/data/answers.txt", "r") as f:
    answers = f.read().splitlines()

with open("/Users/tonypiacentini/csc-306-Project-4/src/data/answers_lite.txt", "r") as f:
    sample_answers = f.read().splitlines()

with open("/Users/tonypiacentini/csc-306-Project-4/src/data/semantics.txt", "r") as f:
    semantics = f.read().splitlines()

# Combine the lists into a DataFrame

# Load the dataset column from the specified file
qaa["answer"] = answers
qaa["sample_answer"] = sample_answers
qaa["type"] = semantics

qa = Dataset.from_pandas(qaa)


In [100]:
print(answers_lite)

['```json\n"Answer":"True"\n```', "'Answer': 'Yes'", '```json\n"Answer":false\n```', '"Answer":"False"', "'Answer': 'False'", "'Answer': 'False'", '```json\n"Answer":"False"\n```', '```json\n"Answer": "True"\n```', "'Answer': 'True'", '```json\n"Answer":"Laboratory Technician"\n```', '```json\n"Answer":"Research & Development"\n```', "'Answer': 'Divorced'", '```json\n"Answer":"Life Sciences"\n```', '```json\n"Answer":"Sales"\n```', '```json\n"Answer":"Female"\n```', '```json\n"Answer":"3"\n```', '```json\n"Answer":"Other"\n```', "'Answer': 37", '```json\n"Answer":5\n```', '```json\n"Answer":31\n```', "'Answer': 2991", '\n  "Answer": 510\n', '```json\n"Answer":31\n```', '```json\n"Answer":4\n```', '```json\n"Answer": 24\n```', '```json\n"Answer":8\n```', "'Answer': '['3', '4', '2', '1']'", '```json\n"Answer": [2028, 2090, 2426, 2661, 2693]\n```', '```json\n"Answer": "[1, 2, 3, 4]"\n```', '\n  "Answer": "[11, 11, 11, 11, 11]"\n', "'Answer': '[41, 49, 37, 33, 27]'", '```json\n"Answer": "[

In [121]:
evaluator = Evaluator(qa=qa)
print(f"DataBench_lite accuracy is {evaluator.eval(reformatted_lite, lite=True)}")


100%|██████████| 522/522 [00:00<00:00, 16720.07it/s]

DataBench_lite accuracy is 0.4674329501915709


In [126]:
myanswers = qaa.drop(columns=['question','answer'])
myanswers['my_answers'] = reformatted_lite
myanswers.head()

,sample_answer,type,my_answers
dataset,,,
066_IBM_HR,True,boolean,True
066_IBM_HR,True,boolean,True
066_IBM_HR,False,boolean,False
066_IBM_HR,False,boolean,False
066_IBM_HR,True,boolean,False
